In [2]:
# import all libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import spacy
nlp= spacy.load("en")

In [3]:
#Load the dataset
train = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", 
                    encoding= "latin-1")
y_train = train[train.columns[0]]
x_train = train[train.columns[5]]

In [5]:
# Split the data into test and train 
from sklearn.model_selection import train_test_split
trainset1x, trainset2x, trainset1y, trainset2y = train_test_split(x_train.values, 
                                                                  y_train.values, 
                                                                  test_size = 0.02, 
                                                                  random_state= 42)
trainset2y= pd.get_dummies(trainset2y)

In [6]:
#function to remove stopwords
def stopwords(sentence):
  new = []
  sentence= nlp(sentence)
  for w in sentence:
    if (w.is_stop == False) & (w.pos_ !="PUNCT"):
      new.append(w.string.strip())
    c=" ".join(str(x) for x in new)
  return c

In [7]:
# funtions to lematize the tweets
def lemmatize(sentence):
  sentence= nlp(sentence)
  str=" "
  for w in sentence:
    str+=" "+w.lemma_
  return nlp(str)

In [24]:
# Loading the glove model
def loadGloveModel(gloveFile):
  print("Loading Glove Model")
  f= open(gloveFile, 'r')
  model = {}
  for line in f:
    splitLine= line.split()
    word = splitLine[0]
    embedding = [float(val) for val in splitLine[1:]]
    model[word] = embedding
  print ("Done."), len(model), (" words loaded!")
  return model

In [25]:
# save the glove model
model = loadGloveModel("/content/glove.twitter.27B.200d.txt")

Loading Glove Model
Done.


In [26]:
# vectorising the sentences
def sent_vectorizer(sent, model):
  sent_vec = np.zeros(200)
  numw = 0
  for w in sent.split():
    try: 
      sent_vec = np.add(sent_vec, model[str(w)])
      numw+=1
    except:
      pass
  return sent_vec

In [27]:
#obtain a clean vector
cleanvector=[]
for i in range(trainset2x.shape[0]):
  document = trainset2x[i]
  document = document.lower()
  document = lemmatize(document)
  document = str(document)
  cleanvector.append(sent_vectorizer(document, model))

In [28]:
# getting the input and output in proper shape
cleanvector = np.array(cleanvector)
cleanvector = cleanvector.reshape(len(cleanvector), 200,1)

In [32]:
#tokenizing the sequences
tokenizer = Tokenizer(num_words=16000)
tokenizer.fit_on_texts(trainset2x)
sequences= tokenizer.texts_to_sequences(trainset2x)
word_index= tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=15, padding="post")
print(data.shape) 

Found 1873 unique tokens.
(400, 15)


In [33]:
# reshape the data and preparing to train
data = data.reshape(len(cleanvector),15,1)
#from sklearn.model_selection import train_test_split
trainx, validx, trainy, validy= train_test_split(data, trainset2y, 
                                                 test_size=0.3, random_state=42)

In [35]:
#calculate the number of words
nb_words= len(tokenizer.word_index)+1

In [37]:
#obtain the embedding matrix
embedding_matrix = np.zeros((nb_words, 200))
for word, i in word_index.items():
  embedding_vector = model.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]= embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis =1) == 0))

trainy= np.array(trainy)
validy= np.array(validy)

Null word embeddings: 655


In [38]:
#building a simple RNN model
def modelbuild():
  model = Sequential()
  model.add(keras.layers.InputLayer(input_shape=(15,1)))
  keras.layers.embeddings.Embedding(nb_words, 15, weights=[embedding_matrix], 
                                    input_length=15, trainable=False)
  
  model.add(keras.layers.recurrent.SimpleRNN(units=100, activation='relu', 
                                             use_bias = True))
  model.add(keras.layers.Dense(units=1000, input_dim=2000, activation='sigmoid'))
  model.add(keras.layers.Dense(units=500, input_dim=1000, activation='relu'))
  model.add(keras.layers.Dense(units=2, input_dim=500, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [39]:
# compiling the model
finalmodel= modelbuild()
finalmodel.fit(trainx, trainy, epochs= 10, batch_size=120, 
               validation_data=(validx,validy))

Train on 280 samples, validate on 120 samples
Epoch 1/10
280/280 [==============================] - 1s 2ms/step - loss: 2.9190 - accuracy: 0.5036 - val_loss: 0.8513 - val_accuracy: 0.5083
Epoch 2/10
280/280 [==============================] - 0s 287us/step - loss: 1.4458 - accuracy: 0.4821 - val_loss: 1.4819 - val_accuracy: 0.4917
Epoch 3/10
280/280 [==============================] - 0s 263us/step - loss: 1.0509 - accuracy: 0.4929 - val_loss: 1.2177 - val_accuracy: 0.5083
Epoch 4/10
280/280 [==============================] - 0s 293us/step - loss: 1.1289 - accuracy: 0.5321 - val_loss: 0.9822 - val_accuracy: 0.5083
Epoch 5/10
280/280 [==============================] - 0s 268us/step - loss: 0.7671 - accuracy: 0.5393 - val_loss: 0.7993 - val_accuracy: 0.4917
Epoch 6/10
280/280 [==============================] - 0s 265us/step - loss: 0.7516 - accuracy: 0.4750 - val_loss: 0.7995 - val_accuracy: 0.4917
Epoch 7/10
280/280 [==============================] - 0s 266us/step - loss: 0.6635 - accurac